<a href="https://colab.research.google.com/github/austiezr/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [x] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [x] Do one-hot encoding of categorical features.
- [x] Do feature selection with `SelectKBest`.
- [x] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [x] Get mean absolute error for the test set.
- [x] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [x] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

## Loading / Cleaning

In [0]:
!pip install category_encoders

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go
import category_encoders as ce
from sklearn.feature_selection import SelectKBest, f_regression
import numpy as np

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

## Splitting / Feature Engineering

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
sub = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
         (df['SALE_PRICE'] > 100000) &
         (df['SALE_PRICE'] < 2000000)]

In [0]:
sub = sub.drop(columns=['ADDRESS', 'BUILDING_CLASS_CATEGORY', 'APARTMENT_NUMBER', 'EASE-MENT'])

sub['SALE_DATE'] = pd.to_datetime(sub['SALE_DATE'])

In [0]:
test = sub[(sub['SALE_DATE'] <= '2019-04-30') & (sub['SALE_DATE'] >= '2019-04-01')]
train = sub[(sub['SALE_DATE'] <= '2019-03-31') & (sub['SALE_DATE'] >= '2019-01-01')]

In [0]:
zip10 = test['ZIP_CODE'].value_counts().index[:10]

test.loc[~test['ZIP_CODE'].isin(zip10), 'ZIP_CODE'] = 'OTHER'

zip10 = train['ZIP_CODE'].value_counts().index[:10]

train.loc[~train['ZIP_CODE'].isin(zip10), 'ZIP_CODE'] = 'OTHER'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
#train['BOROUGH'] = train['BOROUGH'].astype(int)
train['LAND_SQUARE_FEET'] = train['LAND_SQUARE_FEET'].str.replace(',', '').astype(int)

#test['BOROUGH'] = test['BOROUGH'].astype(int)
test['LAND_SQUARE_FEET'] = test['LAND_SQUARE_FEET'].str.replace(',', '').astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### One Hot Encoding

In [0]:
encoder = ce.OneHotEncoder(use_cat_names=True)
train = encoder.fit_transform(train)
test = encoder.transform(test)

## Trying SelectKBest

In [0]:
target = 'SALE_PRICE'
features = train.columns.drop([target, 'SALE_DATE'])

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

with np.errstate(divide='ignore',invalid='ignore'):

  selector = SelectKBest(score_func=f_regression, k=15)

  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

In [0]:
selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

Features selected:
BOROUGH_3
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
BLOCK
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A3
ZIP_CODE_OTHER
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET
GROSS_SQUARE_FEET
BUILDING_CLASS_AT_TIME_OF_SALE_A3


# Regressions -- Best MAE: $152,362.78

## Ridge Regression W/ Select K Best -- Best MAE: $166,307.43

In [0]:
def RidgeRegression(X_train=X_train, y_train=y_train, 
                    X_test=X_test, y_test=y_test, 
                    mae_ref=2000000):
  
  with np.errstate(divide='ignore',invalid='ignore'):

    for k in range(1, len(X_train.columns)+1):
      try:
        mae = mae_ref
      finally:
        selector = SelectKBest(score_func=f_regression, k=k)

        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        model = Ridge(normalize=True)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        mae = mean_absolute_error(y_test, y_pred)
    

        try:
          if mae < mae_ref:
            mae_ref = mae
            final_k = k
        finally: 
          if k == len(X_train.columns):
            print(f'{final_k} Features')
            print(f'Mean Absolute Error: ${mae_ref:,.2f}\n')
            print(X_train_selected.shape)

            selected_mask = selector.get_support()
            all_names = X_train.columns
            selected_names = all_names[selected_mask]
            unselected_names = all_names[~selected_mask]

            print('Features selected:')
            for name in selected_names:
               print(name)


RidgeRegression()

56 Features
Mean Absolute Error: $166,307.43

(2507, 58)
Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
BOROUGH_1
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
BLOCK
LOT
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_CLASS_AT_PRESENT_A4
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_B2
BUILDING_CLASS_AT_PRESENT_S0
BUILDING_CLASS_AT_PRESENT_B3
ZIP_CODE_OTHER
ZIP_CODE_11413.0
ZIP_CODE_11434.0
ZIP_CODE_10312.0
ZIP_CODE_10314.0
ZIP_CODE_11234.0
ZIP_CODE_10304.0
ZIP_CODE_10306.0
ZIP_CODE_10305.0
ZIP_CODE_11412.0
ZIP_CODE_10465.0
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET
GROSS_SQUARE

## Ridge CV Regression W/ Select K Best -- Lowest MAE: $152,362.78

In [36]:
def RidgeCVRegression(X_train=X_train, y_train=y_train, 
                    X_test=X_test, y_test=y_test, 
                    mae_ref=2000000):
  
  with np.errstate(divide='ignore',invalid='ignore'):

    mae_list = []

    for k in range(1, len(X_train.columns)+1):
      try:
        mae = mae_ref
      finally:
        selector = SelectKBest(score_func=f_regression, k=k)

        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        model = RidgeCV(alphas=(np.arange(.1, 100, .1)), normalize=True)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        mae = mean_absolute_error(y_test, y_pred)
    

        try:
          if mae < mae_ref:
            mae_ref = mae
            final_k = k
            mae_list.append(mae)
        finally: 
          if k == len(X_train.columns):
            print(f'{final_k} Features\n')
            print(f'Mean Absolute Error: ${mae_ref:,.2f}\n')
            print(f'Mean MAE: ${np.mean(mae_list):,.2f}\n')

            selected_mask = selector.get_support()
            all_names = X_train.columns
            selected_names = all_names[selected_mask]
            unselected_names = all_names[~selected_mask]

            print('Features selected:')
            for name in selected_names:
               print(name)


RidgeCVRegression()

55 Features

Mean Absolute Error: $152,362.78

Mean MAE: $160,752.86

Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
BOROUGH_1
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
BLOCK
LOT
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_CLASS_AT_PRESENT_A4
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_B2
BUILDING_CLASS_AT_PRESENT_S0
BUILDING_CLASS_AT_PRESENT_B3
ZIP_CODE_OTHER
ZIP_CODE_11413.0
ZIP_CODE_11434.0
ZIP_CODE_10312.0
ZIP_CODE_10314.0
ZIP_CODE_11234.0
ZIP_CODE_10304.0
ZIP_CODE_10306.0
ZIP_CODE_10305.0
ZIP_CODE_11412.0
ZIP_CODE_10465.0
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET